In [111]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
import math

In [144]:
@dataclass
class Config:
    n_layers: int = 2
    d_model: int = 15
    eps: float = 1e-5
    hidden_layer: int = 4 * d_model
    num_heads: int = 3
    
config = Config()
config   

Config(n_layers=2, d_model=15, eps=1e-05, hidden_layer=60, num_heads=3)

In [ ]:
# Layer Normalization layer

class LayerNormalization(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.eps = config.eps
        self.scale = nn.Parameter(torch.ones((config.d_model)))
        self.shift = nn.Parameter(torch.zeros((config.d_model))) 
        
    def forward(self, x: torch.Tensor):
        x_mean = x.mean(dim=-1, keepdim=True) 
        x_std = x.std(dim=-1, keepdim=True)
        x_norm = (x - x_mean) / (x_std + self.eps)
        return x * self.scale + self.shift
                
        
        
        

In [ ]:
# FeedForward Layer
class FeedForwardLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        d_model = config.d_model
        hidden_layer = config.hidden_layer
        
        self.ff_layer = nn.Sequential([
            nn.Linear(d_model, hidden_layer),
            nn.GELU(),
            nn.Linear(hidden_layer, d_model)        
        ])
        
    def forward(self, x):
        return x + self.ff_layer(x)
        
        

In [146]:
class SelfAttentionLayer(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.query_weights = nn.Linear(config.d_model, config.d_model)
        self.key_weights = nn.Linear(config.d_model, config.d_model)
        self.value_weights = nn.Linear(config.d_model, config.d_model)
        self.num_heads = config.num_heads
        assert config.d_model % config.num_heads == 0, "d_model should be divisible by num_heads"
        self.h_dmodel = config.d_model // config.num_heads
        self.neg_inf = - 1e+5
        
    def forward(self, x):
        qeury_vectors = self.query_weights(x)
        key_vectors = self.key_weights(x)
        value_vectors = self.value_weights(x)
        batch_size, seq_len, _ = x.size()
        
        # (B,S,d_model) -> (B, S, num_head, h_dmodel)
        qeury_vectors = qeury_vectors.view(batch_size, seq_len, self.num_heads, self.h_dmodel)
        key_vectors = key_vectors.view(batch_size, seq_len, self.num_heads, self.h_dmodel)
        value_vectors = value_vectors.view(batch_size, seq_len, self.num_heads, self.h_dmodel)
        
        # (B,S, num_heads, h_dmodel) -> (b,num_heads, S, h_dmodel)
        qeury_vectors = torch.permute(qeury_vectors, dims=(0, 2, 1, 3))
        key_vectors = torch.permute(key_vectors, dims=(0, 2, 1, 3))
        value_vectors = torch.permute(value_vectors, dims=(0, 2, 1, 3))
        casual_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool().unsqueeze(0).unsqueeze(0)
        
        attention_scores = self.calculate_attention_score(qeury_vectors, key_vectors, casual_mask)
        contextualized_vectores = attention_scores@value_vectors
        
        contextualized_vectores = torch.permute(contextualized_vectores, dims=(0, 2, 1, 3))
        contextualized_vectores = contextualized_vectores.contiguous().view(batch_size, seq_len, self.num_heads*self.h_dmodel)
        return (contextualized_vectores, attention_scores)
    
    def calculate_attention_score(self, qeury, key, mask):
        # (B,NumHeads,Seq, h_dmodel) * (B,num_heads,h_model, seq) => (B,num_heads, seq, seq)
        k_dmodel = key.size(-1)
        attention_scores = qeury @ key.transpose(-1,-2) / math.sqrt(k_dmodel)
        attention_scores = torch.masked_fill(attention_scores, mask=mask, value=self.neg_inf)
        attention_scores = torch.softmax(attention_scores, dim=-1)
        return attention_scores
        
        
        